In [23]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/NLP

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/NLP


In [24]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import v_measure_score
from sklearn.cluster import KMeans, DBSCAN
import numpy as np
from torch.utils.data import TensorDataset
import torch.nn.functional as F
import random

In [25]:
USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [26]:
data = pd.read_csv("bibledata.csv")

In [27]:
means = torch.load("bible_similar_mean.pt", map_location = device)
embeddings = torch.load("bible_embeddings.pt", map_location = device)

styles = embeddings - means
target = F.normalize(styles)

In [28]:
similarity_matrix = embeddings.matmul(embeddings.T)

In [29]:
top_n = similarity_matrix.sort()[1][:,-7:-1]

In [30]:
random.seed(42)
dataset = TensorDataset(embeddings, target, torch.arange(len(data)).to(device))
train_dataloader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last = True)

In [31]:
l1 = 256
l2 = 128
l3 = 64
z_dim = 32

In [32]:
class L2Norm(nn.Module):
    def __init__(self, dim=1, eps=1e-12):
        super(L2Norm, self).__init__()
        self.dim = dim
        self.eps = eps

    def forward(self, x):
        self.dim = len(x.shape) - 1
        return F.normalize(x, p=2, dim=self.dim, eps=self.eps)

In [33]:
def Encoder(x_dim, z_dim):
  model = nn.Sequential(
    nn.Linear(x_dim, l1),
    nn.ReLU(),
    nn.Linear(l1, l2),
    nn.ReLU(),
    nn.Linear(l2, l3),
    nn.ReLU(),
    nn.Linear(l3, z_dim))
  return model

In [34]:
def Decoder(x_dim, z_dim):
  model = nn.Sequential(
  nn.Linear(z_dim, l3),
    nn.ReLU(),
    nn.Linear(l3, l2),
    nn.ReLU(),
    nn.Linear(l2, l1),
    nn.ReLU(),
    nn.Linear(l1, x_dim),
    L2Norm())
  return model

In [35]:
class Model(nn.Module):
  def __init__(self, x_dim, z_dim, eps = 0.01):
      super(Model, self).__init__()
      self.Encoder = Encoder(x_dim, z_dim)
      self.Decoder = Decoder(x_dim, z_dim)
      self.eps = eps
  
  def forward(self, x):
      x += self.eps*torch.randn(x.shape).to(device)
      z = self.Encoder(x)
      out = self.Decoder(z)
        
      return out

In [36]:
epochs = 50
batch_size = 64

In [37]:
model = Model(384, z_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [38]:
for epoch in tqdm(range(epochs)):
    overall_loss = 0
    for batch_idx, (x, label, index) in enumerate(train_dataloader):
        x = x.view(batch_size, 384)
        nearest = embeddings[top_n[index]]

        optimizer.zero_grad()
        predict = model(x)
        nears = model.Encoder(nearest)

        loss = torch.sum(torch.square(1 - F.cosine_similarity(predict, label))) + torch.sum(torch.square(F.cosine_similarity(model.Encoder(x).unsqueeze(1), nears, dim=2)))/6
      
        overall_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    print("\tEpoch", epoch + 1, "complete!", "\tAverage Loss: ", overall_loss / (batch_idx*64))

  2%|▏         | 1/50 [00:00<00:48,  1.01it/s]

	Epoch 1 complete! 	Average Loss:  1.3445456523493111


  4%|▍         | 2/50 [00:02<00:48,  1.02s/it]

	Epoch 2 complete! 	Average Loss:  0.962127679801849


  6%|▌         | 3/50 [00:04<01:09,  1.48s/it]

...

	Epoch 48 complete! 	Average Loss:  0.4229946301644107


 98%|█████████▊| 49/50 [01:00<00:01,  1.12s/it]

	Epoch 49 complete! 	Average Loss:  0.42074720364019097


100%|██████████| 50/50 [01:01<00:00,  1.23s/it]

	Epoch 50 complete! 	Average Loss:  0.41704668086695384


In [39]:
zs = model.Encoder(embeddings).detach().cpu()
km = KMeans(n_clusters = 7)
km.fit(zs)
data["labels"] = km.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [40]:
v_measure_score(data.id, data.labels)

0.2829570355375063

In [41]:
km.fit(styles.cpu().detach())
data["ll"] = km.labels_
v_measure_score(data.id, data.ll)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0.31213204182992954